# Estudo das Pistas no Parque do Xingu 

In [1]:
#Bibliotecas Usadas
import requests
import re
import geopandas as gpd
from bs4 import BeautifulSoup
from ipyleaflet import Map, basemaps, basemap_to_tiles, TileLayer, GeoData, LegendControl
import pandas as pd

## Importa as coordenadas da Terra Indígena do Parque do Xingu

In [2]:
# Desabilita a verificação de SSL pois o site da FUNAI não ter certificados válidos
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
#Importa o mapa do servidor da Funai
reservas = gpd.read_file('https://geoserver.funai.gov.br/geoserver/Funai/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=Funai:tis_poligonais&CQL_FILTER=dominio_uniao=%27t%27&outputFormat=SHAPE-ZIP')

#Filtra apenas a terra do Parque do Xingu
xingu = reservas[reservas['terrai_nom'] == 'Parque do Xingu']

#Coloca todos os dados em WGS84
xingu_wgs = xingu.to_crs(epsg=4326)

In [4]:
x = float(xingu_wgs.representative_point().y)

In [5]:
x

-11.80442616

## Prepara as pistas cadastradas no Parque do Xingu

In [6]:
# Importa o banco de dados de aeródromos privados
aerod_publicos = pd.read_csv('https://sistemas.anac.gov.br/dadosabertos/Aerodromos/Lista%20de%20aer%C3%B3dromos%20p%C3%BAblicos/AerodromosPublicos.csv',
                             encoding='latin1',
                             header=1,
                             delimiter=';',
                             usecols=['LATGEOPOINT', 'LONGEOPOINT'])

# Importa o banco de dados de aeródromos privados
aerod_privados = pd.read_csv('https://sistemas.anac.gov.br/dadosabertos/Aerodromos/Lista%20de%20aer%C3%B3dromos%20privados/Aerodromos%20Privados/AerodromosPrivados.csv',
                             encoding='latin1',
                             header=1,
                             delimiter=';',
                             usecols=['LATGEOPOINT', 'LONGEOPOINT'])

# Concatena ambos e cria um GeoDataFrame
aerod = pd.concat([aerod_publicos, aerod_privados], ignore_index=True)
aerod_cad = gpd.GeoDataFrame(geometry=gpd.points_from_xy(aerod['LONGEOPOINT'], aerod['LATGEOPOINT']),crs="EPSG:4326")

In [7]:
pistas_pqxingu = gpd.sjoin(aerod_cad, xingu_wgs)

## Prepara as pistas pedidas pelo IBAMA de acordo com a Resolução nº 623/2021

Define uma função para ler as coordenadas das pistas da FUNAI e devolver como um GeoDataFrame

In [8]:
def getlatlon(url):
    
    # Baixa e prepara o html
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, 'html.parser')
    
    # Regex
    prog1 = re.compile("""(N|S)\s?(\d+)\s?°\s?(\d+)\s?\\'\s?(\d+\.?\,?\d*?)\"""")
    lat = prog1.findall(soup.text)

    prog2 = re.compile("""(W|E)\s?(\d+)\s?°\s?(\d+)\s?\\'\s?(\d+\.?\,?\d*?)\"""")
    lon = prog2.findall(soup.text)
    
    # Check de sanidade
    if len(lat) != len(lon):
        print("Erro de Leitura")

    # Constroi a lista de Lat/Lon
    llat = []
    llon = []
    for i in range(len(lat)):
        aux = float(lat[i][1]) + float(lat[i][2])/60.0 + float(lat[i][3])/3600
        if(lat[i][0] == 'S'): aux =-aux;
        llat.append(aux)
        aux = float(lon[i][1]) + float(lon[i][2])/60.0 + float(lon[i][3])/3600
        if(lon[i][0] == 'W'): aux =-aux;
        llon.append(aux)
    
    #Constroi o GeoPandasDataFrame
    gdf = gpd.GeoDataFrame(geometry=gpd.points_from_xy(llon,llat),crs="EPSG:4326")
    
    return gdf

Lê o oficio com as pistas do parque xingu.

TBD - Generalizar para ler outros oficios (precisa ajustar o Regex)

In [9]:
url = 'https://sei.anac.gov.br/sei/modulos/pesquisa/md_pesq_documento_consulta_externa.php?9LibXMqGnN7gSpLFOOgUQFziRouBJ5VnVL5b7-UrE5Q9q65QFzAAEPklK1frAghRMJU7Kj2s9FOJ1vzHUvSiNBMacl-eLe3__9FGUFieg85NA5snk8dZO-H7PYyMb6AG'
pistas_623 = getlatlon(url)

## Pega as Pistas do MapBiomas

In [58]:
# Baixa o banco de dados deles
pistas_mb = gpd.read_file('https://mapbiomas-br-site.s3.amazonaws.com/downloads/Pistas_de_Pouso_02_Fev_2023_v1__3_.zip')

# Filtra as pistas no Parque do Xingu (eles marcam)
pistas_mb_ti = pistas_mb.dropna(subset=['TI'])
pistas_mb_xingu = pistas_mb_ti[pistas_mb_ti['TI'].str.contains('Xingu')]

## Coloca tudo em um gráfico

In [84]:
m = Map(default_tiles=provider, center=(float(xingu_wgs.representative_point().y),float(xingu_wgs.representative_point().x)), zoom=8)
m = Map(center=(float(xingu_wgs.representative_point().y),float(xingu_wgs.representative_point().x)), zoom=8)
m.layout.height = '800px'
legend = LegendControl({}, name="Legenda", position="topright")
m.add_control(legend)

In [79]:
#Adiciona os limites da terra indígena
geo_xingu = GeoData(geo_dataframe = xingu_wgs,
                    style={'color': 'black', 'fillOpacity':0.05},
                    name = 'Parque do Xingu')
legend.add_legend_element("Parque Xingu","black")  # Add a legend element
m.add_layer(geo_xingu)

In [80]:
# Adiciona as pistas do MapBiomas
geo_pistas = GeoData(geo_dataframe = pistas_mb_xingu,
                     style={'color': 'red', 'radius':10, 'fillColor': 'ff3333', 'fillopacity':0.9},
                    point_style={'color': 'red'})
legend.add_legend_element("Pistas MapBiomas","red")  # Add a legend element
m.add_layer(geo_pistas)

In [81]:
## Adiciona as pistas do Cadastrads
geo_pistas = GeoData(geo_dataframe = pistas_pqxingu,
                     style={'color': 'green', 'radius':6, 'fillColor': '52ff33', 'fillopacity':0.9},
                    point_style={'color': 'green'})
legend.add_legend_element("Aeródromos Cadastrados","green")  # Add a legend element
m.add_layer(geo_pistas)

In [82]:
## Adiciona as pistas da Resolução 623
geo_pistas = GeoData(geo_dataframe = pistas_623 ,
                     style={'color': 'blue', 'radius':6, 'fillColor': '3349ff', 'fillopacity':0.9},
                    point_style={'color': 'green'})
legend.add_legend_element("Resolução 623","blue")  # Add a legend element
m.add_layer(geo_pistas)

In [85]:
#Finalmente, mostra o mapa
m

Map(center=[-11.80442616, -53.61171957945864], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoo…